In [1]:
import openpyxl
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
#!pip install geopy

In [2]:
def import_excel_data(file_path, sheet_name):
    # Load the workbook and select the desired sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook[sheet_name]

    # Get labels from row 2 (B2 to G2)
    labels = [sheet.cell(row=2, column=col).value for col in range(2, 7)]  # Columns B-G
    # print(labels)
    
    # Check if all labels are present
    if not all(labels):
        raise ValueError("One or more labels in row 2 are missing.")

    # Initialize a dictionary to store the data for each label
    data = {label: [] for label in labels}
    
    # Get data from rows starting at B3 (columns B-G)
    for row in sheet.iter_rows(min_row=3, min_col=2, max_col=7, values_only=True):
        for col_index, value in enumerate(row):
            if value:  # Skip empty cells
                data[labels[col_index]].append(value)
    return data

In [3]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Startup'  # Replace with the name of the sheet

try:
    data_SU = import_excel_data(file_path, sheet_name)
    print(data_SU)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['LightMatter', 'Lightelligence', 'Ayar Labs', 'PsiQuantum', 'Quantum Transistors', 'Q.ANT', 'Lumai ', 'iPRONICS', 'Salience Labs', 'Pixel Photonics', 'Dream Photonics ', 'Akhetonics', 'Celestial AI', 'Xscape Photonics', 'Optelligence', 'Nubis Communications', 'Voyant Photonics', 'Aegiq', 'Nu-quantum', 'Miraex', 'Nordic Quantum Computing Group', 'Orca Computing', 'Photonic Inc', 'Quandela ', 'Quix', 'Sparrow Quantum', 'Lumiphase', 'Enosemi', 'Nexus Photonics', 'Black Semiconductor'], 'Size': [200, 200, '200+', 280, 50, 100, 20, '< 50', '< 50', '< 50', 10, '< 50', '< 200', '< 50', '< 3', '< 50 ', '< 30', '< 50 ', '< 50 ', 50, 'CLOSED 11/2024', '~100', '< 50', '< 50', '< 50', '< 50 ', '< 100 ', '< 50 ', '< 20 ', '<200'], 'Focus': ['Optical MVM / PNN', 'Optical MVM / Networking', 'Optical I/O for AI, Interconnection ', 'Quantum Computing / Optical Interconnect', 'solid-state\xa0quantum\xa0processor', 'Optical MVM', '3D optical AI computing ', 'Microwave Photonics and Netw

In [4]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Large Company'  # Replace with the name of the sheet

try:
    data_LC = import_excel_data(file_path, sheet_name)
    print(data_LC)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['Nvidia', 'Google ', 'NEC', 'Intel ', 'Universal Display Corporation', 'OpenAI', 'Apple', 'AMD', 'Cisco', 'Luxtera', 'Infinera ', 'NeoPhotonics', 'Coherent Inc. ', 'Hamamatsu Corporation', 'M Squared\xa0', 'NTT', 'IBM', 'Synopsys Photonic Solutions'], 'Size': [500, 100], 'Focus': ['quantum technology, biophotonics, and chemical sensing'], 'CEO/Founders/Affliation': ['GRAEME MALCOLM OBE, Strathclyde University'], 'Location': ['NJ, USA and Japan', 'Glasgow and London and Palo Alto and  Boston and Berlin', 'Zurich, Switzerland', 'Mountain View, California']}


In [5]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Foundries'  # Replace with the name of the sheet

try:
    data_F = import_excel_data(file_path, sheet_name)
    print(data_F)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['Global Foundries', 'AIM ', 'AMF ', 'HHI ', 'Smart Photonics', 'LionX', 'Tower', 'LIGENTEC', 'VTT', 'IMEC'], 'Size': [], 'Focus': [], 'CEO/Founders/Affliation': ['Michael Geiselmann, Michael Zervas,  Tobias Kippenberg,  EPFL '], 'Location': ['Vaud, Switzerland']}


In [6]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Defense Contractors'  # Replace with the name of the sheet

try:
    data_DC = import_excel_data(file_path, sheet_name)
    print(data_DC)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['Bascom Hunter', 'Photonic Systems Inc', 'Phase Sensitive Innovations', 'Honeywell'], 'Size': ['< 100', '< 100', '< 100'], 'Focus': ['Microwave Photonics, Neuromorphioc Photonics', 'Microwave Photonics, Interference Cancellation', 'Microwave Photonics, Diffractive Optics, Phase Arrays, Thin Film LiNo'], 'CEO/Founders/Affliation': ['Paul R. Prucnal , Andy McCandless', 'Charles Cox, Edward Ackerman', 'Dennis Prather, Univ of Delware '], 'Location': ['Baton Rouge, LA']}


### If a company has multiple locations splits that into new rows and then duplicate the information to the new rows, the deliminator is " and "

In [7]:
keyword = " and "

In [8]:
# search and split
def search_and_split(Locations, keyword):
    results = []
    split_strings = []
    NewLocations = []
    Indx2Dup = []
    j=0
    NewLocations = Locations
    for idx, string in enumerate(Locations):
        # print(idx)
        start_idx = string.find(keyword)
        if start_idx != -1:  # Keyword found
            end_idx = start_idx + len(keyword) - 1
            results.append((idx, start_idx, end_idx))

            # Split the string into two parts: before and after the keyword
            before_keyword = string[:start_idx].strip()
            after_keyword = string[end_idx + 1:].strip()
            split_strings.append((before_keyword, after_keyword))
            
            # Append and replace new split string into original list
            # Conditionals are to look for blank spaces or punctions and not include them 
            threshold = 2
            if len(before_keyword) > threshold and len(after_keyword) > threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = before_keyword
                NewLocations.insert(idx+1, after_keyword)
                #idx is no longer real as New List grows, need to keep track
                Indx2Dup.append(idx-j)
                j+=1
                #When duplicating the rows if there are consecutive number dup from 
                    # row - number of consecutive numbers
                
            if len(before_keyword) <= threshold and len(after_keyword) > threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = after_keyword
                
            if len(before_keyword) > threshold and len(after_keyword) <= threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = before_keyword            

                
    return results, split_strings, NewLocations, Indx2Dup


In [9]:
# Duplicate: 
def duplicate_values_with_repeated_indices(indices, source_list):
    # Create a new list to store the results
    result = []

    for i in range(len(source_list)):
        count = indices.count(i)  # Count occurrences of the index in the indices array
        result.extend([source_list[i]] * (count + 1))  # Original + count of duplicates

    return result

In [10]:
# Testing new search and split function
# strings = [
#     "This is a sample string and it contains and perhaps denver and.",
#     "Another example without the keyword.",
#     "and is a conjunction.",
#     "Nothing here or here",
#     "But maybe here and there"
# ]

# # Search for the keyword "and" and split strings
# keyword = "and" # and is in Netherlands hahaha
# matches, splits, nLocations, Indx = search_and_split(strings, keyword)

# # Print results
# print("Keyword Matches (Start and End Indices):")
# for match in matches:
#     print(f"String index: {match[0]}, Start index: {match[1]}, End index: {match[2]}")

# print("\nSplit Strings:")
# for idx, (before, after) in enumerate(splits):
#     print(f"Original String Index: {matches[idx][0]}")
#     print(f"Before: '{before}'")
#     print(f"After: '{after}'")
    
# # print("\nResult")
# # print(nLocations)

print("\nLocation List:")
print(data_SU["Location"])
print(len(data_SU["Location"]))

matches, splits, nLocations, Indx = search_and_split(data_SU["Location"], keyword)

print("\nNew Location List:")
print(nLocations)
print(len(nLocations))

print("\nIndex to Duplicate")
print(Indx)


Location List:
['Mountain View, CA and Toronto, Canada, and Boston, MA', 'Boston, MA', 'San Jose, CA', 'Palo Alto, CA', 'Tel Aviv, Israel and NYC, US', 'Stuttgart, Germany', 'Oxford, England', 'Valencia, Spain', 'Oxford, England', 'Germany', 'Vancover, BC, Canada', 'Berlin, Germany', 'Santa Clara, CA, USA', 'NY, USA', 'Austin, USA', 'Murray Hill, New Jersey, USA', 'New York, NY 10018', 'Sheffield, England', 'Cambridge, UK', 'Écublens, Vaud, Switzerland', 'Oslo, Norway ', 'London, Toronto, Austin', 'Vancouver, Canada', 'Massy, Île-de-France', 'Ulm, Germany and Stuttgart Germany, and Amsterdam, Netherlands, and Enschede, Netherlands', 'Copenhagen, Denmark', 'Zurich, Switzerland', 'California,USA', 'Goleta, California, USA', 'Aachen, Germany']
30

New Location List:
['Mountain View, CA', 'Toronto, Canada,', 'Boston, MA', 'Boston, MA', 'San Jose, CA', 'Palo Alto, CA', 'Tel Aviv, Israel', 'NYC, US', 'Stuttgart, Germany', 'Oxford, England', 'Valencia, Spain', 'Oxford, England', 'Germany', '

In [11]:
# print(data_SU.keys())

In [12]:
# # Example usage
# indices = [1, 1, 3, 3, 3, 5]  # Indices for duplication
# source_list = ["a", "b", "c", "d", "e", "f"]

# # Duplicate values based on repeated indices
# result = duplicate_values_with_repeated_indices(indices, source_list)
# print(result)

In [13]:
# Duplicate values at the specified indices for start up tab
data_SU['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_SU['Company Name'])
data_SU['Size'] = duplicate_values_with_repeated_indices(Indx, data_SU['Size'])
data_SU['Focus'] = duplicate_values_with_repeated_indices(Indx, data_SU['Focus'])
data_SU['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_SU['CEO/Founders/Affliation'])


In [14]:
# Same for LC, F, and DC
matches, splits, nLocations, Indx = search_and_split(data_LC["Location"], keyword)
data_LC['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_LC['Company Name'])
data_LC['Size'] = duplicate_values_with_repeated_indices(Indx, data_LC['Size'])
data_LC['Focus'] = duplicate_values_with_repeated_indices(Indx, data_LC['Focus'])
data_LC['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_LC['CEO/Founders/Affliation'])

matches, splits, nLocations, Indx = search_and_split(data_F["Location"], keyword)
data_F['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_F['Company Name'])
data_F['Size'] = duplicate_values_with_repeated_indices(Indx, data_F['Size'])
data_F['Focus'] = duplicate_values_with_repeated_indices(Indx, data_F['Focus'])
data_F['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_F['CEO/Founders/Affliation'])

matches, splits, nLocations, Indx = search_and_split(data_DC["Location"], keyword)
data_DC['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_DC['Company Name'])
data_DC['Size'] = duplicate_values_with_repeated_indices(Indx, data_DC['Size'])
data_DC['Focus'] = duplicate_values_with_repeated_indices(Indx, data_DC['Focus'])
data_DC['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_DC['CEO/Founders/Affliation'])


### Now I have corrected list for companies with multiple location, we convert rough individual locations into coordinates and make a new column

In [15]:
# Convert Rough Location to longitude and latitude

def get_coordinates_with_duplicates(locations):
    # Initialize the geolocator
    geolocator = Nominatim(user_agent="location_to_coordinates")
    cache = {}  # Cache to store already-queried locations
    results = []  # Final list of results

    for location in locations:
        if location in cache:
            # Use cached result if available
            results.append(cache[location])
        else:
            try:
                # Geocode the location
                loc = geolocator.geocode(location)
                if loc:
                    coordinates = (loc.latitude, loc.longitude)
                else:
                    coordinates = None
                # Cache the result and append to results
                cache[location] = coordinates
                results.append(coordinates)
            except GeocoderTimedOut:
                # Handle timeout gracefully
                results.append(None)

    return results

In [16]:
print(data_SU["Location"])

['Mountain View, CA', 'Toronto, Canada,', 'Boston, MA', 'Boston, MA', 'San Jose, CA', 'Palo Alto, CA', 'Tel Aviv, Israel', 'NYC, US', 'Stuttgart, Germany', 'Oxford, England', 'Valencia, Spain', 'Oxford, England', 'Germany', 'Vancover, BC, Canada', 'Berlin, Germany', 'Santa Clara, CA, USA', 'NY, USA', 'Austin, USA', 'Murray Hill, New Jersey, USA', 'New York, NY 10018', 'Sheffield, England', 'Cambridge, UK', 'Écublens, Vaud, Switzerland', 'Oslo, Norway ', 'London, Toronto, Austin', 'Vancouver, Canada', 'Massy, Île-de-France', 'Ulm, Germany', 'Stuttgart Germany,', 'Amsterdam, Netherlands,', 'Enschede, Netherlands', 'Copenhagen, Denmark', 'Zurich, Switzerland', 'California,USA', 'Goleta, California, USA', 'Aachen, Germany']


In [18]:
# Get coordinates from location list
coordinates_SU = get_coordinates_with_duplicates(data_SU["Location"])

In [19]:
coordinates_LC = get_coordinates_with_duplicates(data_LC["Location"])

In [20]:
coordinates_F = get_coordinates_with_duplicates(data_F["Location"])

In [21]:
coordinates_DC = get_coordinates_with_duplicates(data_DC["Location"])

In [ ]:
# # Example list of locations
# locations = [
#     "Boston, MA, USA",
#     "New York, NY, USA",
#     "Boston, MA, USA",  # Duplicate
#     "San Francisco, CA, USA",
#     "New York, NY, USA"  # Duplicate
# ]

# # Get coordinates
# coordinates = get_coordinates_with_duplicates(locations)

# # Print results
# for location, coord in zip(locations, coordinates):
#     if coord:
#         print(f"{location}: Latitude = {coord[0]}, Longitude = {coord[1]}")
#     else:
#         print(f"{location}: Coordinates not found.")